In [1]:
from collections import Counter
from time import time
import numpy as np

In [38]:
def evaluate_performance(trials = 1):
    filename = 'C:/workspace/PYTHON/data/random_forest_data.txt'
    data = np.loadtxt(filename, delimiter = ',')  # (267, 45)
    X = data[:, 1:]
    y = data[:, 0]
    n, d = X.shape   # n = 행 , d = 열 (267, 44)
    print('input_data\n', X)
    print('input_data\n', y)
    print('input_data_set_shape', n, d)
    print('label_data_set_shape', n, d)
    
    accuracy_list = np.zeros(trials)
    
    for trial in range(trials):
        print('##    Trial    ##', trial)
        
        # Random Shuffle
        idx = np.arange(n)
        print('idx\n', idx)   # idx list
        np.random.seed(np.int(time() / 150))
        np.random.shuffle(idx)  # X, y data set shuffle
        X = X[idx]
        Y = y[idx]
        
        # Spilt Train and Test samples ( train(.8) / test(.2))
        train = np.int(0.8 * n)
        Xtrain = X[:train, :]  # (213, 44)
        Xtest = X[train:, :]
        Ytrain = Y[:train]
        Ytest = Y[train:]
        print('Xtrain\n', Xtrain)
        print('Ytrain\n', Ytrain)
        
        tr_n, tr_d = Xtrain.shape
        te_n, te_d = Xtest.shape
        
        print('Xtrain_shape : ', tr_n, tr_d)
        print('Xtest_shape : ', te_n, te_d)
        
        # Setting parameter for make forest
        clf = RandomForest(n_trees = 3, max_depth = 10, ratio_per_tree = 0.7, ratio_features = 0.3)
        clf.fit(Xtrain, Ytrain)
        
        print('')
        print('##    Test Start    ##\n')
        
        pred = clf.predict(Xtest)
        accuracy_list[trial] = accuracy_score(Ytest, pred)
        
    stats = np.zeros((3, 3))
    stats[0, 0] = np.mean(accuracy_list)
    
    return stats

In [39]:
# Make Forest
class RandomForest(object):
    def __init__(self, n_trees=10, max_depth=10, ratio_per_tree=0.5, ratio_features=1.0):
        self.trees = None
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.ratio_per_tree = ratio_per_tree
        self.ratio_features = ratio_features

    def fit(self, X, Y):
        n, d = X.shape
        print('Forest_data_shape', n, d)
        print('Num of tree : ', self.n_trees)
        self.trees = []  # contain tree

        # Maek tree
        for i in range(self.n_trees):
            idx = np.arange(n)
            # print(idx)
            # np.random.seed(np.int(time() / 100))
            np.random.shuffle(idx)
            X = X[idx]
            Y = Y[idx]
            train = np.int(self.ratio_per_tree * n)
            print('train(ratio_per_tree * n) : ', train)
            Xtrain = X[:train, :]
            print('tree_train_data', Xtrain)
            Ytrain = Y[:train]
            print('tree_label_train_data', Ytrain)
            
            clf = RandomTree(max_depth=self.max_depth, ratio_features=self.ratio_features)
            # print('for_tree_max_depth : ', self.max_depth)
            # print('for_tree_ratio_features : ', self.ratio_features)
            # print('forest_ratio_feaures', self.ratio_features)
            clf.fit(Xtrain, Ytrain)  # 객체 생성
            # print('forest_shape', Xtrain.shape)
            self.trees.append(clf)
            # print('tress', self.trees)
            
    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees]).T
        print('predictions 1 : ', predictions)
        predictions = np.array([np.argmax(np.bincount(prediction)) for prediction in predictions])
        print('predictions 2 : ', predictions)
        
        return predictions

In [40]:
class DecisionNode(object):
    def __init__(self,
                 feature=None,
                 value=None,
                 false_branch=None,
                 true_branch=None,
                 is_leaf=False,
                 current_results=None
                 ):
        self.feature = feature
        self.value = value
        self.false_branch = false_branch
        self.true_branch = true_branch
        self.is_leaf = is_leaf
        self.current_results = current_results

In [41]:
class RandomTree(object):
    def __init__(self, max_depth=10, ratio_features=1.0):
        self.tree = None
        self.max_depth = max_depth
        self.ratio_features = ratio_features
        # self.get_split_feature(self)
        # self.split_feature_list = []

    def fit(self, X, Y):
        print('forest_ratio_feaures', self.ratio_features)
        Y = Y.reshape((1, X.shape[0])).T
        print('reshape_y', Y)
        data = np.concatenate((X, Y), axis=1)
        print('tree_data\n', data)
        print('tree_data_shape\n', data.shape)
        self.tree = build_tree(data, ratio_features=self.ratio_features, max_depth=self.max_depth)

    def predict(self, X):
        predict_show = np.array([pred(self.tree, x) for x in X])
        print('predict_show  : ', predict_show) # tree 개수 만큼 출력된다.
        
        return np.array([pred(self.tree, x) for x in X])

In [42]:
def label_frequencies(data):
    n, d = data.shape
    labels = data[:, (d - 1):].reshape(n).astype(int)
    return np.bincount(labels)


def different_labels(labels):
    c = 0
    for label in labels:
        if label != 0:
            c += 1
    return c

In [43]:

def split_data(data, feature, value):  # data = tree data set(56,5), featuer : index, value : feature value
    t, f = [], []
    for row in data:
        if row[feature] >= value:
            t.append(row)
        else:
            f.append(row)

    return np.array(t), np.array(f)

In [44]:
def entropy(data):
    n = len(data)
    if n == 0:
        return 0
    labels = label_frequencies(data).astype(float) / n
    # print('Prb_label_select : ', labels)
    e = .0
    for label in labels:
        if label > .0:
            e -= label * np.log2(label)
#     print('labels_entropy : ', e)

    return e

def entropy_for_feature(data, feature):  # feature는 feature를 뽑은 index가 저장된다.
    values = np.unique(data[:, feature])  # values 에는 feature 값들이 저장된다.
    # print('entropy_feature_values\n', values)
    entropy_loss = 1e5
    value = 0
    for val in values:
        t, f = split_data(data, feature, val)
        # print('true : ', t)
        # print('false : ', f)
        # print('t_entropy', t)
        curr_entropy_loss = (len(t) * entropy(t) + len(f) * entropy(f)) / len(data)
        print('curr_entropy_loss : ', curr_entropy_loss)
        if entropy_loss > curr_entropy_loss: # entropy loss가 가장 작은 value를 추출한다.
            entropy_loss = curr_entropy_loss
            value = val
    print('entropy_for_featrue : ',entropy_loss)
    return entropy_loss, value  # entropy loss가 가장 작은 feature value 값과 entropy값 return

In [45]:
split_feature_list = []


def get_split_feature(fea):
    split_feature_list.append(fea)
    


    return split_feature_list

In [46]:
def build_tree(data, depth=0, ratio_features=1.0, max_depth=100):
    n = len(data)
    if n == 0:
        return DecisionNode(is_leaf=True)
    if n != 0:
        n, d = data.shape

    current_results = label_frequencies(data)  # result : [ number_of_0 , unmber_of_1]
    if depth == max_depth:  # 나무의 depth가 max_depth에 도달했거나 하나의 클래스만 남았을 경우 노드를 종료한다.
        return DecisionNode(current_results=current_results, is_leaf=True)
    if different_labels(current_results) == 1:
        return DecisionNode(current_results=current_results, is_leaf=True)

    # Select Random Features
    seed = np.int(time() / 150)
    idx = np.arange(d - 1)
    np.random.seed(seed)
    np.random.shuffle(idx)
    idx = idx[: np.int(ratio_features * (d - 1))]
    print('selected_idx\n', idx)

    entropy_loss = 1e5
    feature = None
    value = None

    for feat in idx:
        print('chekc_feat : ', feat)
        entropy, val = entropy_for_feature(data, feat)
        if entropy_loss > entropy:
            entropy_loss = entropy
            feature = feat
            value = val
            split_feature = get_split_feature(feature) # importance variable을 구하기 위해 split에 사용된 feature를 저장한다.
            print('split_feature : ', split_feature)
            cnt = Counter(split_feature) 
            print('count_split_feature\n', cnt)

#         if len(split_feature) == np.int(ratio_features * (d - 1)) + 1: # tree개수 만큼 feature가 저장되면 초기화한다.
#             split_feature = []

    print('select_feature_list : ', split_feature_list)
    print('Best for split feature : ', feature)
    print('Best for split value : ', value)
    print('')
    print('###  real tree split start  ###')
    t, f = split_data(data, feature, value) # entropy loss가 가장 낮은 feature를 가지고 child node를 만들어 나간다.    
    print('false_branch_shape', f.shape)
    print('true_branch_shape', t.shape)


    return DecisionNode(

        feature=feature,
        value=value,
        false_branch=build_tree(f, depth + 1, ratio_features, max_depth),
        true_branch=build_tree(t, depth + 1, ratio_features, max_depth),
        current_results=current_results
    )

In [47]:

def accuracy_score(Y_true, Y_predict):
    Y_true = Y_true.astype(int)
    t = .0
    for i in range(np.size(Y_true)):
        if Y_true[i] == Y_predict[i]:
            t += 1
    return t / np.size(Y_predict)

In [48]:
def pred(tree, x):
    if tree.is_leaf or different_labels(tree.current_results) == 1:
        return np.argmax(tree.current_results)
    else:
        if x[tree.feature] >= tree.value:
            return pred(tree.true_branch, x)
        else:
            return pred(tree.false_branch, x)

In [49]:
if __name__ == '__main__':
    stats = evaluate_performance()
    print('Accuracy= ', stats[0, 0])

input_data
 [[59. 52. 70. ... 74. 64. 67.]
 [72. 62. 69. ... 71. 56. 58.]
 [71. 62. 70. ... 41. 51. 46.]
 ...
 [75. 73. 72. ... 75. 67. 71.]
 [59. 62. 72. ... 76. 70. 70.]
 [64. 66. 68. ... 64. 57. 54.]]
input_data
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.

C:\Users\CPB06G~1\AppData\Local\Temp/ipykernel_12892/651787565.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.random.seed(np.int(time() / 150))
C:\Users\CPB06G~1\AppData\Local\Temp/ipykernel_12892/651787565.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the relea

curr_entropy_loss :  0.6823394114984858
curr_entropy_loss :  0.6817615282117794
curr_entropy_loss :  0.6809936449585715
curr_entropy_loss :  0.6790312614053953
curr_entropy_loss :  0.6778832465282255
curr_entropy_loss :  0.6766450441466143
curr_entropy_loss :  0.6739399908814754
curr_entropy_loss :  0.6709791355785513
curr_entropy_loss :  0.6781166848896674
curr_entropy_loss :  0.6759871414676928
curr_entropy_loss :  0.6709537725555696
curr_entropy_loss :  0.6712447408056418
curr_entropy_loss :  0.6744216050057547
curr_entropy_loss :  0.6709888610880251
curr_entropy_loss :  0.6748265712563591
curr_entropy_loss :  0.6759539436527823
curr_entropy_loss :  0.6758815966524331
curr_entropy_loss :  0.6818911199323179
curr_entropy_loss :  0.6813880516409394
curr_entropy_loss :  0.6798635040644898
curr_entropy_loss :  0.6826690897103457
curr_entropy_loss :  0.6776625490027222
curr_entropy_loss :  0.6791249063395618
curr_entropy_loss :  0.675882225314417
curr_entropy_loss :  0.6802634969932198
c

curr_entropy_loss :  0.6552585977648431
entropy_for_featrue :  0.6173137412725129
chekc_feat :  17
curr_entropy_loss :  0.6722948170756379
curr_entropy_loss :  0.6552585977648431
curr_entropy_loss :  0.6369953778417903
curr_entropy_loss :  0.6173137412725129
curr_entropy_loss :  0.5959734049880651
curr_entropy_loss :  0.5184064113121721
curr_entropy_loss :  0.6603738884808443
curr_entropy_loss :  0.5469917825204
curr_entropy_loss :  0.5726669113829173
curr_entropy_loss :  0.6369953778417903
entropy_for_featrue :  0.5184064113121721
chekc_feat :  24
curr_entropy_loss :  0.6722948170756379
curr_entropy_loss :  0.6552585977648431
curr_entropy_loss :  0.6369953778417903
curr_entropy_loss :  0.6173137412725129
curr_entropy_loss :  0.6711711490715918
curr_entropy_loss :  0.6720327836423284
curr_entropy_loss :  0.6603738884808443
curr_entropy_loss :  0.6682935528794068
curr_entropy_loss :  0.5469917825204
curr_entropy_loss :  0.5726669113829173
curr_entropy_loss :  0.6173137412725129
curr_ent

curr_entropy_loss :  0.8417363179213259
curr_entropy_loss :  0.8569211525841409
curr_entropy_loss :  0.8624652045311298
curr_entropy_loss :  0.8590051121588616
curr_entropy_loss :  0.862951144813532
curr_entropy_loss :  0.8561775704944491
curr_entropy_loss :  0.8587041770320938
curr_entropy_loss :  0.8545538983312201
curr_entropy_loss :  0.8610966428873729
curr_entropy_loss :  0.8630300223737775
curr_entropy_loss :  0.8517616236370373
curr_entropy_loss :  0.8434746247918015
curr_entropy_loss :  0.8628323217750904
curr_entropy_loss :  0.8573003913349806
entropy_for_featrue :  0.8417363179213259
chekc_feat :  0
curr_entropy_loss :  0.863120568566631
curr_entropy_loss :  0.8573003913349806
curr_entropy_loss :  0.8513960314759039
curr_entropy_loss :  0.8454049929637689
curr_entropy_loss :  0.839324665933993
curr_entropy_loss :  0.8531873831444232
curr_entropy_loss :  0.8600132161815653
curr_entropy_loss :  0.8614189617184125
curr_entropy_loss :  0.8562908071827299
curr_entropy_loss :  0.86

...

curr_entropy_loss :  0.9216964181919894
curr_entropy_loss :  0.9295553174587589
entropy_for_featrue :  0.9165643512915066
chekc_feat :  37
curr_entropy_loss :  0.9372930661967526
curr_entropy_loss :  0.9295553174587589
curr_entropy_loss :  0.9216964181919894
curr_entropy_loss :  0.9137124482750648
curr_entropy_loss :  0.905599290864763
curr_entropy_loss :  0.897352618782833
curr_entropy_loss :  0.9358912582210772
curr_entropy_loss :  0.9334406764018627
curr_entropy_loss :  0.9301124490217391
curr_entropy_loss :  0.9312039608286313
curr_entropy_loss :  0.9342118532731412
curr_entropy_loss :  0.9369991317315871
curr_entropy_loss :  0.9343030909556762
curr_entropy_loss :  0.9359111928165358
curr_entropy_loss :  0.9343094841909896
curr_entropy_loss :  0.9341595952378636
curr_entropy_loss :  0.9354293717058986
curr_entropy_loss :  0.9352548286413206
curr_entropy_loss :  0.9289923268542996
curr_entropy_loss :  0.9308315895165963
curr_entropy_loss :  0.912201291806694
curr_entropy_loss :  0.9

curr_entropy_loss :  0.9722610255677093
curr_entropy_loss :  0.9700202140346532
curr_entropy_loss :  0.9719097918458721
curr_entropy_loss :  0.9515101408070366
entropy_for_featrue :  0.9515101408070366
chekc_feat :  0
curr_entropy_loss :  0.972552316568843
curr_entropy_loss :  0.9621286787420126
curr_entropy_loss :  0.9515101408070366
curr_entropy_loss :  0.940689095326383
curr_entropy_loss :  0.9681878345798915
curr_entropy_loss :  0.9625137761370411
curr_entropy_loss :  0.9712102470547711
curr_entropy_loss :  0.968053134941976
curr_entropy_loss :  0.9710286738745659
curr_entropy_loss :  0.9671976812339866
curr_entropy_loss :  0.9724997775547077
curr_entropy_loss :  0.9718553898274777
curr_entropy_loss :  0.970556004391738
curr_entropy_loss :  0.9601973751000534
curr_entropy_loss :  0.9706974805350878
curr_entropy_loss :  0.9723643088021936
curr_entropy_loss :  0.9690384133862224
curr_entropy_loss :  0.9671976812339866
curr_entropy_loss :  0.9720730699929665
curr_entropy_loss :  0.971

curr_entropy_loss :  0.6896596952239762
curr_entropy_loss :  0.6877840558577583
entropy_for_featrue :  0.6041843979966417
chekc_feat :  17
curr_entropy_loss :  0.7219280948873623
curr_entropy_loss :  0.6877840558577583
curr_entropy_loss :  0.6490224995673063
curr_entropy_loss :  0.6041843979966417
curr_entropy_loss :  0.7145247027726656
curr_entropy_loss :  0.6877840558577583
entropy_for_featrue :  0.6041843979966417
chekc_feat :  24
curr_entropy_loss :  0.7219280948873623
curr_entropy_loss :  0.4529325012980812
curr_entropy_loss :  0.6348515545596771
curr_entropy_loss :  0.6896596952239762
curr_entropy_loss :  0.7145247027726656
curr_entropy_loss :  0.6896596952239762
entropy_for_featrue :  0.4529325012980812
chekc_feat :  27
curr_entropy_loss :  0.7219280948873623
curr_entropy_loss :  0.6877840558577583
curr_entropy_loss :  0.6490224995673063
curr_entropy_loss :  0.6041843979966417
curr_entropy_loss :  0.7145247027726656
curr_entropy_loss :  0.7219280948873623
curr_entropy_loss :  0.

curr_entropy_loss :  0.6490224995673063
entropy_for_featrue :  0.5509775004326938
chekc_feat :  0
curr_entropy_loss :  0.9709505944546686
curr_entropy_loss :  0.8
curr_entropy_loss :  0.5509775004326938
curr_entropy_loss :  0.9509775004326937
curr_entropy_loss :  0.6490224995673063
entropy_for_featrue :  0.5509775004326938
select_feature_list :  [10, 11, 43, 27, 14, 10, 30, 10, 43, 27, 26, 10, 10, 10, 11, 37, 27, 10, 11, 37, 10, 11, 37, 17, 10, 15, 10, 10, 24, 10, 11, 10, 30, 11, 10, 30, 10, 30, 15, 10, 10, 34, 26, 10, 30, 0, 10, 10, 11, 10, 30, 10, 43, 10, 34, 10, 43, 24, 14, 10, 11, 43, 10, 30, 11, 27, 10, 30, 11, 24, 27, 10, 34, 15, 10, 14, 10, 11, 37, 10, 10, 11, 10, 24, 10, 30, 37, 10, 30, 27, 10, 30, 10, 10, 34, 43, 10, 34, 10, 30, 10, 30, 34, 17, 24, 0, 10, 30, 11, 14, 10, 30, 10, 34, 10, 30, 11, 43, 10, 30, 43, 15, 14, 10, 30, 11, 17, 14, 10, 30, 11, 34, 10, 14, 10, 11, 37, 24, 10, 11, 37, 10, 17, 10, 37, 24, 10, 14, 10, 10, 17, 26, 10, 43, 17, 10, 30, 17, 10, 24]
Best for spli

curr_entropy_loss :  0.9389462162661898
curr_entropy_loss :  0.9241743523004412
curr_entropy_loss :  0.9389462162661898
curr_entropy_loss :  0.937101105625982
curr_entropy_loss :  0.9152077851647805
curr_entropy_loss :  0.9397964894836078
curr_entropy_loss :  0.8925768472426705
entropy_for_featrue :  0.839887505700988
chekc_feat :  24
curr_entropy_loss :  0.9402859586706311
curr_entropy_loss :  0.8925768472426705
curr_entropy_loss :  0.839887505700988
curr_entropy_loss :  0.7810237375890372
curr_entropy_loss :  0.7142857142857143
curr_entropy_loss :  0.8949517866414867
curr_entropy_loss :  0.8500096092778661
curr_entropy_loss :  0.8921589282623617
curr_entropy_loss :  0.8380423950607804
curr_entropy_loss :  0.8268850944895277
entropy_for_featrue :  0.7142857142857143
split_feature :  [10, 11, 43, 27, 14, 10, 30, 10, 43, 27, 26, 10, 10, 10, 11, 37, 27, 10, 11, 37, 10, 11, 37, 17, 10, 15, 10, 10, 24, 10, 11, 10, 30, 11, 10, 30, 10, 30, 15, 10, 10, 34, 26, 10, 30, 0, 10, 10, 11, 10, 30, 1